In [1]:
import struct
import os


In [2]:
# 拼音表偏移，
startPy = 0x1540;

# 汉语词组表偏移
startChinese = 0x2628;

# 全局拼音表
GPy_Table = {}

# 解析结果
# 元组(词频,拼音,中文词组)的列表
GTable = []

# 原始字节码转为字符串
def byte2str(data):
    pos = 0
    str = ''
    while pos < len(data):
        c = chr(struct.unpack('H', bytes([data[pos], data[pos + 1]]))[0])
        if c != chr(0):
            str += c
        pos += 2
    return str

# 获取拼音表
def getPyTable(data):
    data = data[4:]
    pos = 0
    while pos < len(data):
        index = struct.unpack('H', bytes([data[pos],data[pos + 1]]))[0]
        pos += 2
        lenPy = struct.unpack('H', bytes([data[pos], data[pos + 1]]))[0]
        pos += 2
        py = byte2str(data[pos:pos + lenPy])

        GPy_Table[index] = py
        pos += lenPy

# 获取一个词组的拼音
def getWordPy(data):
    pos = 0
    ret = ''
    while pos < len(data):
        index = struct.unpack('H', bytes([data[pos], data[pos + 1]]))[0]
        ret += GPy_Table[index]
        pos += 2
    return ret

# 读取中文表
def getChinese(data):
    pos = 0
    while pos < len(data):
        # 同音词数量
        same = struct.unpack('H', bytes([data[pos], data[pos + 1]]))[0]

        # 拼音索引表长度
        pos += 2
        py_table_len = struct.unpack('H', bytes([data[pos], data[pos + 1]]))[0]

        # 拼音索引表
        pos += 2
        py = getWordPy(data[pos: pos + py_table_len])

        # 中文词组
        pos += py_table_len
        for i in range(same):
            # 中文词组长度
            c_len = struct.unpack('H', bytes([data[pos], data[pos + 1]]))[0]
            # 中文词组
            pos += 2
            word = byte2str(data[pos: pos + c_len])
            # 扩展数据长度
            pos += c_len
            ext_len = struct.unpack('H', bytes([data[pos], data[pos + 1]]))[0]
            # 词频
            pos += 2
            count = struct.unpack('H', bytes([data[pos], data[pos + 1]]))[0]

            # 保存
            GTable.append((count, py, word))

            # 到下个词的偏移位置
            pos += ext_len


def scel2txt(file_name):
    print('-' * 60)
    with open(file_name, 'rb') as f:
        data = f.read()

    print("词库名：", byte2str(data[0x130:0x338])) # .encode('GB18030')
    print("词库类型：", byte2str(data[0x338:0x540]))
    print("描述信息：", byte2str(data[0x540:0xd40]))
    print("词库示例：", byte2str(data[0xd40:startPy]))

    getPyTable(data[startPy:startChinese])
    getChinese(data[startChinese:])


In [3]:
# scel所在文件夹路径
in_path = "../chusai/dict/"
# 输出词典所在文件夹路径
out_path = "user_dict.txt"

fin = [fname for fname in os.listdir(in_path) if fname[-5:] == ".scel"]
for f in fin:
    f = os.path.join(in_path, f)
    scel2txt(f)

# 保存结果
with open(out_path, 'w', encoding='utf8') as f:
    f.writelines([word+'\n' for count, py, word in GTable])


------------------------------------------------------------
词库名： TFBOYS粉丝专用【官方推荐】
词库类型： 明星
描述信息： Hi，粉丝们！小编整理了你们喜欢的偶像的昵称、演唱歌曲、参加节目等相关词条，让你追星so easy~
词库示例： 王俊凯 四叶草 千纸鹤 想唱就唱 青春修炼手册 男生学院自习室 
------------------------------------------------------------
词库名： 日剧电影动漫和日本明星
词库类型： 明星
描述信息： 日剧、日本电影、日本动漫和日本明星
 黑崎彩春 上原绫香
------------------------------------------------------------
词库名： 乒乓球【官方推荐】
词库类型： 乒乓球
描述信息： 官方推荐，词库来源于网友上传！
 世界排名 五夹 乒乓球网
------------------------------------------------------------
词库名： 足球明星
词库类型： 足球
描述信息： 关于足球明星名字的一个词库
 李云在尔森罗纳尔多
------------------------------------------------------------
词库名： 饮食大全【官方推荐】
词库类型： 饮食
描述信息： 官方推荐，词库来源于网友上传！
 国珍 ： 蜜柑
------------------------------------------------------------
词库名： 全国省市区县地名
词库类型： 单位机构名
描述信息： 比搜狗自带的还全！！！
词库示例： 澳门 重庆 福建 河北 黑龙江 江西 
------------------------------------------------------------
词库名： 服装设计专用词库【官方推荐】
词库类型： 服饰
描述信息： 本词库专为服装设计专业的同学们私人订制，为小伙伴们学习、生活助力，打字完全无阻碍！
词库示例： 树脂衬 捆条 顺风褶 平面裁剪 斜纹布 定型带 
----------------------------------------

 阿桑芬安又琪戴
------------------------------------------------------------
词库名： 手机词汇大全【官方推荐】
词库类型： 日常
描述信息： 官方推荐，词库来源于网友上传！
 大唐通信机操作系统 处理器主频
------------------------------------------------------------
词库名： 魔兽世界【官方推荐】
词库类型： 角色扮演
描述信息： 官方推荐，词库来源于网友上传！
词库示例： 座狼毛皮短裤 座狼血液药剂 作战之刃 作战指令  座狼的毛皮 
------------------------------------------------------------
词库名： 2018年世界杯球迷专用【官方推荐】
词库类型： 足球
描述信息： 世界杯正在火热进行中，球友们都pick哪个队呢？（别告诉我你pick中国队）不管你是真球迷还是伪球迷，词库君都为你们准备了一份大礼，专业术语、球星名字应有尽有，让你痛快看比赛，畅快聊起来！妈妈再也不用担心我打不出字啦
词库示例： 倒挂金钩  清道夫 梅开二度 全攻全守 补位 
------------------------------------------------------------
词库名： 新番动漫人名库
词库类型： 新番
描述信息： 包括目前新番中的动漫人名
 花园花铃界国崎往人
------------------------------------------------------------
词库名： 篮球术语词库
词库类型： 篮球
描述信息： 本人搜集了一些关于篮球方面的词汇，欢迎大家修正添加。
 跳投离投篮钩射投篮
------------------------------------------------------------
词库名： 生物词汇大全【官方推荐】
词库类型： 普通生物学
描述信息： 官方推荐，词库来源于网友上传！
 深肌腱： 肝着
------------------------------------------------------------
词库名： 韩国明星
词库类型： 明星
描述信息： 一些现在比较当红或是大家比较熟知的韩国明星。。。由于是音译的